## Imports

In [2]:
import pandas as pd
import numpy as np
import glob
import os
import seaborn as sns
from seaborn import plt

from __future__ import division, unicode_literals
import re
import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity


%matplotlib inline

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/veenakumar/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [1]:
import pickle

In [17]:
with open("./review_matrix_aggreg_final.pkl", 'rb') as picklefile: 
    review_index = pickle.load(picklefile)
    
with open("./model_glove_final.pkl", 'rb') as picklefile: 
    model = pickle.load(picklefile)

In [42]:
import re, string
from string import maketrans
from gensim.models import Word2Vec

In [43]:
glove_path = '/Users/veenakumar/ds/metis/metisgh/sf16_ds4/local/Projects/5Kojak/glove.6B/glove.6B.300d.txt'

In [44]:
model = gensim.models.Word2Vec.load_word2vec_format(glove_path) 

In [6]:
reviews = pd.read_csv("./reviews_clean_FINAL.csv",index_col=0)
listings = pd.read_csv("./listings_FINAL.csv",index_col=0)

In [7]:
reviews.head()

,listing_id,comments
0,1417398,Catherine and Gabe were away when we arrived. ...
1,1417398,"We had a warm welcome from Catherine, Gabe and..."
2,1417398,Great room. It's nice to share a space but sti...
3,1417398,The room was perfect for our stay with a priva...
4,1417398,Everything was perfect! Catherine and Gabe wer...


In [12]:
listings.head()

,name,id,bedrooms,bathrooms,accommodates,description,listing_url,latitude,longitude,neighbourhood_cleansed,number_of_reviews,picture_url,price,property_type,review_scores_rating,summary,thumbnail_url,xl_picture_url,zipcode
0,Lovely One Bedroom Apartment,1162609,1.0,1.0,2,This is a big well designed space. Our neighbo...,https://www.airbnb.com/rooms/1162609,37.785217,-122.488655,Seacliff,8,https://a2.muscache.com/ac/pictures/104807019/...,350.0,Apartment,89.0,This is a big well designed space. Our neighbo...,https://a2.muscache.com/ac/pictures/104807019/...,https://a2.muscache.com/ac/pictures/104807019/...,94121.0
1,"Cozy Vintage Room, Golden Gate Park",8568925,1.0,1.0,2,Darling room in the heart of a gorgeous SF nei...,https://www.airbnb.com/rooms/8568925,37.774604,-122.450261,Haight Ashbury,1,https://a1.muscache.com/ac/pictures/109547397/...,92.0,Apartment,100.0,Darling room in the heart of a gorgeous SF nei...,https://a1.muscache.com/ac/pictures/109547397/...,https://a1.muscache.com/ac/pictures/109547397/...,94117.0
2,Sunny Top Floor Family Apt - 3 bdr,288380,3.0,1.5,6,"Our apt has been built with love, and all of o...",https://www.airbnb.com/rooms/288380,37.772882,-122.439946,Haight Ashbury,0,https://a2.muscache.com/ac/pictures/96b9a9d0-a...,420.0,Apartment,NaN,"Our apt has been built with love, and all of o...",https://a2.muscache.com/ac/pictures/96b9a9d0-a...,https://a2.muscache.com/ac/pictures/96b9a9d0-a...,94117.0
3,Clean + Quiet + Convenient,8762798,1.0,1.0,3,Clean and quiet (Im rarely home) + large open ...,https://www.airbnb.com/rooms/8762798,37.770441,-122.450553,Haight Ashbury,3,https://a2.muscache.com/ac/pictures/111006173/...,120.0,Apartment,100.0,Clean and quiet (Im rarely home) + large open ...,https://a2.muscache.com/ac/pictures/111006173/...,https://a2.muscache.com/ac/pictures/111006173/...,94117.0
4,Elegant Room in Victorian Mansion,8428014,1.0,2.0,2,Beautiful master bedroom with lofted ceiling o...,https://www.airbnb.com/rooms/8428014,37.772116,-122.449435,Haight Ashbury,2,https://a2.muscache.com/ac/pictures/107194427/...,180.0,Apartment,90.0,Beautiful master bedroom with lofted ceiling o...,https://a2.muscache.com/ac/pictures/107194427/...,https://a2.muscache.com/ac/pictures/107194427/...,94117.0


In [49]:
review_vecs = get_doc_vecs(model, reviews.comments)

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


Failed Doc!


In [51]:
vecs = np.empty((len(review_vecs), 300))
for i in range(len(review_vecs)):
    vecs[i, : ] = review_vecs[i]

In [50]:
from gensim.similarities import MatrixSimilarity 

In [52]:
review_index = MatrixSimilarity(gensim.matutils.Dense2Corpus(vecs.transpose()))

In [5]:
def get_doc_vecs(model, docs):
    stoplist = stopwords.words('english')
    words_counted = 0
    words_skipped = 0
    docs_vecs = []
    for doc in docs:
        doc_vecs = []
        try:
            regex = re.compile('[%s]' % re.escape(string.punctuation))
            doc = doc.lower().split()
            for word in doc:
                if word not in stoplist:
                    try:
                        word = str(word)
                        word = regex.sub('',word)
                        doc_vecs.append(model[word])
                        words_counted += 1
                    except:
                        words_skipped += 1
                        pass
            docs_vecs.append(np.array(doc_vecs).mean(axis=0))
        except:
            print "Failed Doc!"
            pass
    return docs_vecs

In [61]:
def airbnb_experience(review_index,model,user_input,reviews_df,listings_df):
    
    #obtaining top thousand comments closest to user input
    scores = [score for score in review_index[get_doc_vecs(model,user_input)[0]]]
    indexes = range(0,len(review_vecs))
    topthousand = sorted(zip(scores, indexes), reverse=True)[:1000]

    #getting the indexes and vectors
    dfindexes = [tup[1] for tup in topthousand]
    dfvectors = [tup[0] for tup in topthousand]

    #creating a results df
    results = reviews_df.iloc[dfindexes]
    results['vectors'] = dfvectors

    #sorting the results so highest listings are on top
    results_sorted = results.groupby('listing_id').sum().sort('vectors',ascending=False)
    results_sorted.reset_index(inplace=True)
    results_sorted.listing_id = results_sorted.listing_id.map(int)

    #new subset df with listing information for the top listings
    results_listings = listings[listings['id'].isin(results_sorted.listing_id.astype(int))]

    #merging both df's so the listings df can be sorted by vectors descending
    final = pd.merge(results_listings,results_sorted,left_on='id',right_on='listing_id').sort('vectors',ascending=False)
    return final

## Tests

### Test 1

In [66]:
user_input1 = ['near lots of bars and public transportation. cozy beds and great for one or two people.']
test1 = airbnb_experience(review_index,model,user_input1,reviews,listings)

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [67]:
test1.head()

,name,id,bedrooms,bathrooms,accommodates,description,listing_url,latitude,longitude,neighbourhood_cleansed,...,picture_url,price,property_type,review_scores_rating,summary,thumbnail_url,xl_picture_url,zipcode,listing_id,vectors
511,Classic Russian Hill flat in SF!,734839,2.0,1.0,6,"This charming Russian Hill flat, in an Edwardi...",https://www.airbnb.com/rooms/734839,37.801526,-122.409400,North Beach,...,https://a2.muscache.com/ic/pictures/10382187/a...,330.0,Apartment,98.0,"This charming Russian Hill flat, in an Edwardi...",https://a2.muscache.com/ic/pictures/10382187/a...,https://a2.muscache.com/ic/pictures/10382187/a...,94109.0,734839,6.122040
508,CHARMING NORTH BEACH ALLEY HIDEAWAY,1178260,2.0,1.5,4,CHARMING NORTH BEACH ALLEY HIDEAWAY In the hea...,https://www.airbnb.com/rooms/1178260,37.801543,-122.401358,North Beach,...,https://a2.muscache.com/ic/pictures/17888339/f...,250.0,Apartment,99.0,NaN,https://a2.muscache.com/ic/pictures/17888339/f...,https://a2.muscache.com/ic/pictures/17888339/f...,94133.0,1178260,6.116649
384,Beautiful Upscale House Private BR,38607,1.0,1.0,2,Enjoy one of the absolute best and upscale nei...,https://www.airbnb.com/rooms/38607,37.781975,-122.458092,Presidio Heights,...,https://a1.muscache.com/ic/pictures/844895/8f1...,139.0,House,96.0,NaN,https://a1.muscache.com/ic/pictures/844895/8f1...,https://a1.muscache.com/ic/pictures/844895/8f1...,94118.0,38607,6.115148
249,GORGEOUS FLAT IN GREAT LOCATION,42572,3.0,1.5,6,I am a painter and my work takes up most of my...,https://www.airbnb.com/rooms/42572,37.762218,-122.425424,Mission,...,https://a0.muscache.com/ic/pictures/238885/ba6...,360.0,Apartment,98.0,I am a painter and my work takes up most of my...,https://a0.muscache.com/ic/pictures/238885/ba6...,https://a0.muscache.com/ic/pictures/238885/ba6...,94110.0,42572,5.265322
93,Large central 3BR/2BA with parking,930955,3.0,2.0,7,Our home is located almost right in the middle...,https://www.airbnb.com/rooms/930955,37.759827,-122.448205,Haight Ashbury,...,https://a1.muscache.com/ic/pictures/23505927/9...,500.0,Apartment,92.0,Our home is located almost right in the middle...,https://a1.muscache.com/ic/pictures/23505927/9...,https://a1.muscache.com/ic/pictures/23505927/9...,94114.0,930955,5.257318


In [71]:
#obtaining top thousand comments closest to user input
scores = [score for score in review_index[get_doc_vecs(model,user_input1)[0]]]
indexes = range(0,len(review_vecs))
topthousand = sorted(zip(scores, indexes), reverse=True)[:1000]

#getting the indexes and vectors
dfindexes = [tup[1] for tup in topthousand]
dfvectors = [tup[0] for tup in topthousand]

#creating a results df
results = reviews.iloc[dfindexes]
results['vectors'] = dfvectors


/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [76]:
for c in results.comments[:20]:
    print c + "\n"

We stayed in Lenore's apartment for two nights.  We had three adults and two small children, and it worked out fine, with the use of the air mattress for one adult.  We brought along a baby crib for one of the children.  The apartment is clean and nicely cared for, artistic, and has great views.  The kitchen was great to cook in.  The apt. is secure, and located in a good neighborhood in the Mission District.  It is close to public transport and the walkability is great.  There is a playground nearby.  The only difficulty we faced was parking, but Lenore made sure we were aware of the streets nearby and the hours in which parking is prohibited.  Once we caught onto the parking situation and routine, it was only a matter of moving the car a couple of times, to avoid ticketing.  Staying in this apartment was much easier than staying in hotel rooms.  I would recommend it.  For small families, it is nice to be able to put the kids to sleep and spend time as adults, together in the dining a

### Test 2

In [77]:
user_input2 = ['i want to be near local favorites. i like to cook so a large kitchen and something close to a farmers market would be great. i also want to be close to public transportation']
test2 = airbnb_experience(review_index,model,user_input2,reviews,listings)

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [85]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 25 to 285
Data columns (total 21 columns):
name                      613 non-null object
id                        613 non-null int64
bedrooms                  611 non-null float64
bathrooms                 604 non-null float64
accommodates              613 non-null int64
description               613 non-null object
listing_url               613 non-null object
latitude                  613 non-null float64
longitude                 613 non-null float64
neighbourhood_cleansed    613 non-null object
number_of_reviews         613 non-null int64
picture_url               613 non-null object
price                     613 non-null float64
property_type             612 non-null object
review_scores_rating      611 non-null float64
summary                   477 non-null object
thumbnail_url             550 non-null object
xl_picture_url            550 non-null object
zipcode                   611 non-null float64
listing_id      

In [87]:
test2[test2['id']==1898486]

,name,id,bedrooms,bathrooms,accommodates,description,listing_url,latitude,longitude,neighbourhood_cleansed,...,picture_url,price,property_type,review_scores_rating,summary,thumbnail_url,xl_picture_url,zipcode,listing_id,vectors
148,Private Cottage in Noe Valley,1898486,2.0,1.5,4,"Originally built in the 1915 as a horse barn, ...",https://www.airbnb.com/rooms/1898486,37.744497,-122.42718,Noe Valley,...,https://a1.muscache.com/ic/pictures/28535032/d...,235.0,House,100.0,"Originally built in the 1915 as a horse barn, ...",https://a1.muscache.com/ic/pictures/28535032/d...,https://a1.muscache.com/ic/pictures/28535032/d...,94131.0,1898486,3.675032


In [88]:
test2.head()

,name,id,bedrooms,bathrooms,accommodates,description,listing_url,latitude,longitude,neighbourhood_cleansed,...,picture_url,price,property_type,review_scores_rating,summary,thumbnail_url,xl_picture_url,zipcode,listing_id,vectors
25,Moscone center 3 bedrooms Market St,45900,3.0,1.0,6,Located on 1000 block of market st... St 7 th ...,https://www.airbnb.com/rooms/45900,37.782198,-122.410017,South of Market,...,https://a0.muscache.com/ic/pictures/7195853/b0...,240.0,Apartment,85.0,Located on 1000 block of market st... St 7 th ...,NaN,NaN,94103.0,45900,9.094724
476,Secluded Studio in Castro/Mission,1407881,0.0,1.0,2,Welcome to our studio! This is a fully furnish...,https://www.airbnb.com/rooms/1407881,37.763613,-122.430629,Castro/Upper Market,...,https://a1.muscache.com/ic/pictures/21166845/0...,160.0,Apartment,97.0,Welcome to our studio! This is a fully furnish...,https://a1.muscache.com/ic/pictures/21166845/0...,https://a1.muscache.com/ic/pictures/21166845/0...,94114.0,1407881,7.254423
138,Virtually Your Own Apt in Nth Beach,208831,1.0,1.0,2,Classic San Francisco home (Edwardian) in the ...,https://www.airbnb.com/rooms/208831,37.802421,-122.414402,Russian Hill,...,https://a2.muscache.com/ic/pictures/1990594/98...,145.0,House,96.0,Classic San Francisco home (Edwardian) in the ...,https://a2.muscache.com/ic/pictures/1990594/98...,https://a2.muscache.com/ic/pictures/1990594/98...,94133.0,208831,6.376990
186,"Private, cozy studio near nature!",431632,1.0,1.0,2,"Come stay in your own artist-decorated, peacef...",https://www.airbnb.com/rooms/431632,37.781354,-122.482656,Outer Richmond,...,https://a1.muscache.com/ic/pictures/5660657/cd...,140.0,Apartment,91.0,Please note that during this summer (July-Augu...,https://a1.muscache.com/ic/pictures/5660657/cd...,https://a1.muscache.com/ic/pictures/5660657/cd...,94121.0,431632,5.447978
112,Remodeled Cozy Sunny Room! - RED,2047469,1.0,2.0,2,Location: We are located in the San Francisco...,https://www.airbnb.com/rooms/2047469,37.724915,-122.435155,Outer Mission,...,https://a0.muscache.com/ic/pictures/30923347/1...,79.0,House,92.0,Location: We are located in the San Francisco...,https://a0.muscache.com/ic/pictures/30923347/1...,https://a0.muscache.com/ic/pictures/30923347/1...,94112.0,2047469,4.576652


In [79]:
scores = [score for score in review_index[get_doc_vecs(model,user_input2)[0]]]
indexes = range(0,len(review_vecs))
topthousand = sorted(zip(scores, indexes), reverse=True)[:1000]

#getting the indexes and vectors
dfindexes = [tup[1] for tup in topthousand]
dfvectors = [tup[0] for tup in topthousand]

#creating a results df
results2 = reviews.iloc[dfindexes]
results2['vectors'] = dfvectors

/Users/veenakumar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [80]:
for c in results2.comments[:20]:
    print c + "\n"

What a great place, and a great neighborhood as well.  My wife and I, as well as two friends, spent a week here.  It's right on a  public transit line that takes you pretty much wherever you need to go, very economically.  Excellent local shopping with a great grocer and butcher right here.  Easy to cook for yourself if you like, with a well-supplied kitchen.  The guest house is in back of the residence, obviously new construction, modern appliances, very private.  Highly recommended.

Brenda was very welcoming and willing to meet me earlier than normal check-in time. She had lists of local favorite restaurants for me and a map and umbrella for me to use.  The house was beautiful and looked exactly like the pictures. The neighborhood is great, there is a lot of food and shopping close by. There is Whole Food, Safeway, and Bi-Rite close by for anything you might need in addition to many small retail stores and boutiques. There is ample public transportation near by and I was able to get

In [162]:
example = test2[test2['id']==4956697]

In [163]:
columns = ['name','bedrooms','bathrooms','accommodates','description','listing_url']
for i,c in enumerate(columns):
    print example.iloc[i],"\n"

name                                         Sunny 2br/1Ba Garden Apartment
id                                                                  4956697
bedrooms                                                                  2
bathrooms                                                                 1
accommodates                                                              3
description               Beautiful 2br/1ba apartment with access to wel...
listing_url                            https://www.airbnb.com/rooms/4956697
latitude                                                            37.7787
longitude                                                          -122.476
neighbourhood_cleansed                                       Inner Richmond
number_of_reviews                                                        29
picture_url               https://a1.muscache.com/ic/pictures/64243391/7...
price                                                                   119
property_typ

IndexError: single positional indexer is out-of-bounds

In [122]:
for i,v in enumerate(results2.comments[:10]):
    print "Listing ID: ",results2.listing_id.iloc[i],"\n","Comments: ",v,"\n","Vector: ",results2.vectors.iloc[1],"\n","*****************************************"

Listing ID:  1898486 
Comments:  What a great place, and a great neighborhood as well.  My wife and I, as well as two friends, spent a week here.  It's right on a  public transit line that takes you pretty much wherever you need to go, very economically.  Excellent local shopping with a great grocer and butcher right here.  Easy to cook for yourself if you like, with a well-supplied kitchen.  The guest house is in back of the residence, obviously new construction, modern appliances, very private.  Highly recommended. 
Vector:  0.932150363922 
*****************************************
Listing ID:  12522 
Comments:  Brenda was very welcoming and willing to meet me earlier than normal check-in time. She had lists of local favorite restaurants for me and a map and umbrella for me to use.  The house was beautiful and looked exactly like the pictures. The neighborhood is great, there is a lot of food and shopping close by. There is Whole Food, Safeway, and Bi-Rite close by for anything you m

In [171]:
comments = results2[results2['listing_id']==1898486]

In [172]:
for i,v in enumerate(comments.comments[:10]):
    print "Listing ID: ",comments.listing_id.iloc[i],"\n","Comments: ",v,"\n","Vector: ",comments.vectors.iloc[1],"\n","*****************************************"

Listing ID:  1898486 
Comments:  What a great place, and a great neighborhood as well.  My wife and I, as well as two friends, spent a week here.  It's right on a  public transit line that takes you pretty much wherever you need to go, very economically.  Excellent local shopping with a great grocer and butcher right here.  Easy to cook for yourself if you like, with a well-supplied kitchen.  The guest house is in back of the residence, obviously new construction, modern appliances, very private.  Highly recommended. 
Vector:  0.925220608711 
*****************************************
Listing ID:  1898486 
Comments:  Ed and Sunny made us feel very welcome and went out of their way to make our stay as enjoyable as possible. They were very friendly and helpful. The information pack was thorough and informative, containing lots of local knowledge. The cottage was finished to an extremely high standard with quality white goods and fittings. The addition of a washer/dryer was invaluable on a

### Test 3

In [ ]:
user_input3 = ['']
test3 = airbnb_experience(review_index,model,user_input3,reviews,listings)

In [ ]:
scores = [score for score in review_index[get_doc_vecs(model,user_input3)[0]]]
indexes = range(0,len(review_vecs))
topthousand = sorted(zip(scores, indexes), reverse=True)[:1000]

#getting the indexes and vectors
dfindexes = [tup[1] for tup in topthousand]
dfvectors = [tup[0] for tup in topthousand]

#creating a results df
results3 = reviews.iloc[dfindexes]
results3['vectors'] = dfvectors

In [ ]:
for c in results3.comments[:20]:
    print c + "\n"

### Test 4

In [ ]:
user_input4 = ['']
test4 = airbnb_experience(review_index,model,user_input4,reviews,listings)

In [ ]:
scores = [score for score in review_index[get_doc_vecs(model,user_input4)[0]]]
indexes = range(0,len(review_vecs))
topthousand = sorted(zip(scores, indexes), reverse=True)[:1000]

#getting the indexes and vectors
dfindexes = [tup[1] for tup in topthousand]
dfvectors = [tup[0] for tup in topthousand]

#creating a results df
results4 = reviews.iloc[dfindexes]
results4['vectors'] = dfvectors

In [ ]:
for c in results4.comments[:20]:
    print c + "\n"

### Test 5

In [ ]:
user_input5 = ['']
test5 = airbnb_experience(review_index,model,user_input5,reviews,listings)

In [ ]:
scores = [score for score in review_index[get_doc_vecs(model,user_input5)[0]]]
indexes = range(0,len(review_vecs))
topthousand = sorted(zip(scores, indexes), reverse=True)[:1000]

#getting the indexes and vectors
dfindexes = [tup[1] for tup in topthousand]
dfvectors = [tup[0] for tup in topthousand]

#creating a results df
results5 = reviews.iloc[dfindexes]
results5['vectors'] = dfvectors

In [ ]:
for c in results5.comments[:20]:
    print c + "\n"

### Test 6

In [ ]:
user_input = ['']
test6 = airbnb_experience(review_index,model,user_input,reviews,listings)

### Test 7

In [ ]:
user_input = ['']
test7 = airbnb_experience(review_index,model,user_input,reviews,listings)

### Test 8

In [ ]:
user_input = ['']
test8 = airbnb_experience(review_index,model,user_input,reviews,listings)

### Test 9

In [ ]:
user_input = ['']
test9 = airbnb_experience(review_index,model,user_input,reviews,listings)

### Test 10

In [ ]:
user_input = ['']
test10 = airbnb_experience(review_index,model,user_input,reviews,listings)